# Audio Noise Separation
This is part of audio data preprocessing. In this notebook, we will split the audio files into signal part where bird sounds are audible and noise part where there are no bird sounds (background noise may be present in this part). The separation of the audio file into signal and noise parts is done as described in **Sprengel, E., Jaggi, M., Kilcher, Y., & Hofmann, T. (2016). Audio based bird species identification using deep learning techniques**.

## Signal extraction
To separate an audio file into a signal and a noise part, we first compute the spectrogram of the whole file. We then normalize the spectrogram by dividing every pixel with the maximum pixel value. To select the parts that correspond to the signal part, we pick all the pixels that are three times bigger than the row median and three times bigger than the column median. Ideally, this should give us the parts that correspond to a signal since high amplitude to a bird audio. The selected pixels are then set to 1 and everything else to 0. A binary erosion and dilation filter is then applied to get rid of noise and join the segments. A 4 by 4 filter was used in this case.

We then proceed to create an indicator vector with as many elements as there are columns in the spectrogram. The i-th element in this vector is set to 1 if the i-th column contains at least one 1, otherwise it is set to 0. The indicator vector is smoothened by applying two more binary 4 by 1 dilation filters. The indicator vector is then scaled to the length of the original audio file and used as a mask to extract the signal part. The parts obtained are joined to form one signal audio and saved as a signal file.

## Noise extraction
For noise part extraction, we follow the same procedure used above but in this case we select the pixels that are 2.5 times bigger than the row and column median. We then follow the procedure as described above but the results are inverted in the very end. The parts obtained are joined to form one noise audio and saved as a noise file. The use of the different thresholds (3 and 2.5) when selecting the signal and noise parts provides a safety margin in the selection process and it means some columns may not correspond to either sound or noise parts.

In [ ]:
#import necessary libraries
import os
import random
import librosa
import argparse
import warnings
import numpy as np
import configparser
from tqdm import tqdm
import IPython.display
import librosa.display
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.ndimage import binary_dilation, binary_erosion

### Loading parameters needed to compute spectrograms

In [ ]:
warnings.filterwarnings('ignore')

# Get parameters from configuration file
config = configparser.ConfigParser()
config.read('parameters.ini')

win_len_ms = int(config['audio']['win_len_ms'])
overlap = float(config['audio']['overlap'])
sampling_rate = int(config['audio']['sampling_rate'])


# Derive audio processing values
win_len = int((win_len_ms * sampling_rate) / 1000)
hop_len = int(win_len * (1 - overlap))
nfft = int(2 ** np.ceil(np.log2(win_len)))

### Load and play a sample file
Let's load a sample audio file that we will use for the entire of this notebook.

In [ ]:
path = './sample_data'
file = os.path.join(path, os.listdir(path)[0])
y, _ = librosa.load(file, sr=sampling_rate)
IPython.display.Audio(y, rate=sampling_rate)

### Compute and normalize a spectrogram
Now let us compute the spectrogram of the loaded file, normalize it and visualize it.

In [ ]:
specgram = np.abs(librosa.stft(y, 
                    n_fft=nfft, 
                    hop_length=hop_len))
normalized_specgram = specgram / (specgram.max() + 1e-8)

librosa.display.specshow(librosa.amplitude_to_db(specgram, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

### 1. Signal extraction

#### i. Selecting pixels corresponding to signal
We will begin with selecting pixels of normalized spectrogram that are that are 3 times greater than the row and column median and set them all to 1 and the rest to zero.

In [ ]:
#select segments containing audio and plot the spectrogram
threshold = 3
col_mask = specgram > threshold * np.median(specgram, axis=0) #elements of spectrogram greater than 3*median of column
row_mask = specgram.T > threshold * np.median(specgram, axis=1) #elements of spectrogram greater than 3*median of row
row_mask  = row_mask.T
mask = col_mask & row_mask

mask = mask.astype(int)
librosa.display.specshow(librosa.amplitude_to_db(mask, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

#### ii. Binary erosion
Let's apply a 4 by 4 binary erosion filter to the masked spectrogram to remove noise and visualize the output of the filter.

In [ ]:
# erosion
audio_be_mask = binary_erosion(mask, np.ones((4, 4)))
audio_be_mask
audio_be_mask = audio_be_mask.astype(int)
librosa.display.specshow(librosa.amplitude_to_db(audio_be_mask, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

#### iii. Binary dilation
Let's now apply a 4 by 4 binary dilation filter to the spectrogram obtained above to smoothen it visualize the output of the filter.

In [ ]:
# dilation
audio_bd_be_mask = binary_dilation(audio_be_mask, np.ones((2, 2)))
audio_bd_be_mask = audio_bd_be_mask.astype(int)
librosa.display.specshow(librosa.amplitude_to_db(audio_bd_be_mask, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

#### iv. Indicator vector
We will generate an indicator vector of the same length as the number of columns of the spectrogram. Each column of the spectrogram represent a window that the audio file was split into to perform short time fourier transform (STFT). The indicator vector is created by making its i-th element to 1 if there is at least one 1 in the i-th column and 0 otherwise. We then smoothen the vector by applying two 4 by 1 dilation filters.

In [ ]:
selected_col = np.max(audio_bd_be_mask, axis=0)
bd_sel_col = binary_dilation(selected_col[:, None], np.ones((4, 1)))
bd2_sel_col = binary_dilation(bd_sel_col, np.ones((4, 1)))

#### v. Signal mask
The indicator vector obtained above is scaled to the length of the audio file and used to extract parts of the audio that correspond to a signal

In [ ]:
# translate to audio samples
selection_mtx = np.ones((specgram.shape[1], hop_len)) * bd2_sel_col

audio_indx = selection_mtx.flatten().astype(bool)
selection_mtx.shape

audio_indx = selection_mtx.flatten().astype(bool)

signal = y[audio_indx[:len(y)]]
IPython.display.Audio(signal, rate=sampling_rate)

#### vi. Visualizing the extracted signal

In [ ]:
S_audio = np.abs(librosa.stft(signal, 
                        n_fft=nfft, 
                        hop_length=hop_len))
librosa.display.specshow(librosa.amplitude_to_db(S_audio, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

### 2. Noise extraction
In the sections that follow, we will repeat the steps followed in signal extraction, however, we will set the threshold for selecting pixels at 2.5 time greater than row and column median.
#### i. Selecting pixels 2.5 time greater than row and column median.

In [ ]:
threshold = 2.5
col_mask = specgram > threshold * np.median(specgram, axis=0) #elements of spectrogram greater the 3*median of column
row_mask = specgram.T > threshold * np.median(specgram, axis=1) #elements of spectrogram greater the 3*median of row
row_mask  = row_mask.T
mask = col_mask & row_mask

mask = mask.astype(int)
librosa.display.specshow(librosa.amplitude_to_db(mask, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

#### ii. Binary erosion

In [ ]:
# erosion
noise_be_mask = binary_erosion(mask, np.ones((2, 2)))
noise_be_mask
noise_be_mask = noise_be_mask.astype(int)
librosa.display.specshow(librosa.amplitude_to_db(noise_be_mask, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

#### iii. Binary dilation

In [ ]:
# dilation
noise_bd_be_mask = binary_dilation(noise_be_mask, np.ones((2, 2)))
noise_bd_be_mask = noise_bd_be_mask.astype(int)
librosa.display.specshow(librosa.amplitude_to_db(noise_bd_be_mask, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

#### iv. Indicator vector

In [ ]:
selected_col = np.max(audio_bd_be_mask, axis=0)
bd_sel_col = binary_dilation(selected_col[:, None], np.ones((4, 1)))
bd2_sel_col = binary_dilation(bd_sel_col, np.ones((4, 1)))

#### v. Indicator mask
The indicator vector obtained above is scaled to the length of the audio file, inverted and used to extract parts of the audio that correspond to noise.

In [ ]:
# translate to audio samples
selection_mtx = np.ones((specgram.shape[1], hop_len)) * bd2_sel_col

audio_indx = selection_mtx.flatten().astype(bool)
selection_mtx.shape

audio_indx = selection_mtx.flatten().astype(bool)

audio_indx = ~audio_indx

noise = y[audio_indx[:len(y)]]
IPython.display.Audio(noise, rate=sampling_rate)

In [ ]:
S_noise = np.abs(librosa.stft(noise, 
                        n_fft=nfft, 
                        hop_length=hop_len))
librosa.display.specshow(librosa.amplitude_to_db(S_noise, ref=np.max),
                         sr=sampling_rate,
                         hop_length=hop_len,
                         y_axis='linear', 
                         x_axis='time')

Let's now repeat the above steps on the entire files that we downloaded and save the extracted signal and noise parts.

In [ ]:
#These functions get the indices corresponding to audio and noise in a file

def compute_audio_mask(norm_specgram, hop_len, category='audio'):
    """ Compute the section of signal corresponding to audio or noise
    This follows the approach described in
    Sprengel, E., Jaggi, M., Kilcher, Y., & Hofmann, T. (2016).
    Audio based bird species identification using deep learning techniques
    Args:
        norm_specgram: input spectrogram with values in range [0,1]
        hop_len: hop length used to generate the spectrogram
        category: whether 'audio' or 'noise'
    Returns:
        mask: the mask of samples belonging to 'category'
    Raises: ValueError if the category is not known
    """

    if category == 'audio':
        threshold = 3
    elif category == 'noise':
        threshold = 2.5
    else:
        raise ValueError('Unknown category')

    col_mask = norm_specgram > threshold * np.median(norm_specgram, axis=0)
    row_mask = norm_specgram.T > threshold * np.median(norm_specgram, axis=1)
    row_mask  = row_mask.T
    mask = col_mask & row_mask

    # erosion
    be_mask = binary_erosion(mask, np.ones((4, 4)))

    # dilation
    bd_be_mask = binary_dilation(be_mask, np.ones((4, 4)))

    bd_be_mask = bd_be_mask.astype(int)
    selected_col = np.max(bd_be_mask, axis=0)
    bd_sel_col = binary_dilation(selected_col[:, None], np.ones((4, 1)))
    bd2_sel_col = binary_dilation(bd_sel_col, np.ones((4, 1)))


    # translate to audio samples
    selection_mtx = np.ones((norm_specgram.shape[1], hop_len)) * selected_col[:, None]

    audio_indx = selection_mtx.flatten().astype(bool)

    if category == 'audio':
        return audio_indx
    else:
        return ~audio_indx



def get_audio_noise(audio_array, nfft, hop_len):
    """ Get both the signal and noise
    Args:
        audio_array: an array of audio
        nfft: FFT length
        hop_len: hop length
    Returns:
        signal and noise
    """

    specgram = np.abs(librosa.stft(audio_array, n_fft=nfft, hop_length=hop_len))
    specgram_norm = specgram / (specgram.max() + 1e-8)

    audio_indx = compute_audio_mask(specgram_norm, hop_len)[:len(audio_array)]
    noise_indx = compute_audio_mask(specgram_norm, hop_len, 'noise')[:len(audio_array)]


    return audio_array[audio_indx], audio_array[noise_indx]


def audio_noise_save(rec_dir, sr, nfft, hop_len, audio_dir, noise_dir):
    """ Save separated audio and noise parts
    Args:
        audio_array: an array of audio
        nfft: FFT length
        hop_len: hop length
        audio_dir-path to save audio segment
        noise_dir-path to save noise segment
        rec_dir-recordings' directory
        sr-sampling rate
    """
    
    
    birds_dir = next(os.walk(rec_dir))[1]
    for species in birds_dir:
        recs = os.listdir(os.path.join(rec_dir, species))
        path = os.path.join(audio_dir, species) #path to store recording of a given species
        if not os.path.exists(path):
            os.makedirs(path)
        for file in tqdm(recs):
            try:
                y,_ = librosa.load(os.path.join(rec_dir, species, file), sr=sr)
                audio, noise = get_audio_noise(y, nfft, hop_len)
            except Exception as e:
                return
            
            sf.write(os.path.join(path,
                          file.replace('mp3', 'wav')),
             audio,
             sr)
            sf.write(os.path.join(noise_dir,
                                  file.replace('mp3', 'wav')),
                     noise,
                     sr)

In [ ]:
audio_noise_save('./xeno-canto',
                 sr,
                 nfft,
                 hop_len,
                 './noiseless-xeno-canto',
                 './noise')